* https://www.kaggle.com/competitions/nbme-score-clinical-patient-notes/data
* https://www.kaggle.com/datasets/datatattle/covid-19-nlp-text-classification
* https://www.kaggle.com/datasets/clmentbisaillon/fake-and-real-news-dataset/code?datasetId=572515&sortBy=voteCount
* https://www.kaggle.com/code/avikumart/nlp-news-articles-classif-wordembeddings-rnn
* https://www.kaggle.com/code/rcushen/topic-modelling-with-lsa-and-lda

Seperate models for each prediction?

Feed predictions from previous models and append and then feed to next model?

LSTM and GAN

Use a pre-existing model as text are complex

Todo:

Fix Line Break \r\n in Articles column

Task 1: Find articles that are similar, main idea is to use a web crawler and if the article is similar, input that article into phase 2

* Percentage based output, treat it as a classification

Task 2: Predict basic columns first such as date of event, state, district, date of report, villages, country, number of casualties(elephant/humans)

In [1]:
import pandas as pd
#import spacy_transformers
import numpy as np
#import tensorflow as tf
#import tensorflow_hub as hub
#import tensorflow_text as text  # Imports TF ops for preprocessing.
import matplotlib.pyplot as plt
import seaborn as sns
import gc
from sklearn.model_selection import train_test_split
import nltk
from nltk.collocations import *
#print("Version: ", tf.__version__)
#print("Eager mode: ", tf.executing_eagerly())
#print("Hub version: ", hub.__version__)
#print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")


# Loading data and removing Nan's

In [3]:
df = pd.read_csv("data/STE HEC - Master 10.0.2 - Elephant Death & Injury(1).csv", sep=',')
#df.to_excel("data.xlsx")
pd.set_option('max_colwidth', 400)
df.columns

Index(['Article  ID', 'Datefixed', 'Date (Event)', 'Time', 'Village/Place',
       'Sub-District', 'Beat', 'Range', 'Division', 'District', 'State',
       'Country', 'Latitude (DD)', 'Longitude (DD)', 'Herd', 'All male',
       'Total', 'Alcohol', 'Captive?', 'Injured Male', 'Injured Female',
       'Einjured', 'Killed Male', 'Killed Female', 'Edeath', 'Name', 'Age',
       'Gender', 'Alcohol.1', 'Cause of Death/Injury', 'Accidental?',
       'During HEC Encounter', 'Retaliatory', 'Poaching', 'Human Death/Injury',
       'Attractant', 'Area (Forest, Ag, Village, Habitation, Urban)',
       'Legal Electric Fence', 'Fence Broken', 'Elephant Proof Trenches ',
       'Drive away Elephants ', 'Guns', 'Loud Sounds', 'Fine', 'Fine Amount',
       'Jail Sentence', 'Other', 'Comments', 'Links', 'Source', 'Title',
       'Date Published', 'Article', 'No. of Articles'],
      dtype='object')

In [29]:
df.isnull().sum(axis = 0)

Article  ID                                        0
Datefixed                                          0
Date (Event)                                       0
Time                                               0
Village/Place                                      0
Sub-District                                       0
Beat                                               0
Range                                              0
Division                                           0
District                                           0
State                                              0
Country                                            0
Latitude (DD)                                      0
Longitude (DD)                                     0
Herd                                               0
All male                                           0
Total                                              0
Alcohol                                            0
Captive?                                      

In [30]:
df.dropna(subset=['Date Published', 'Article', 'No. of Articles', 'Area (Forest, Ag, Village, Habitation, Urban)'], inplace=True)

In [31]:
df.dtypes

Article  ID                                       object
Datefixed                                         object
Date (Event)                                      object
Time                                              object
Village/Place                                     object
Sub-District                                      object
Beat                                              object
Range                                             object
Division                                          object
District                                          object
State                                             object
Country                                           object
Latitude (DD)                                     object
Longitude (DD)                                    object
Herd                                              object
All male                                          object
Total                                             object
Alcohol                        

# Constructing training set to mimic data that scrapes from the internet

Assuming that when article is scraped from the internet, only date of article published, the publisher, and the article is left.

How to deal with line breaks(\r\n)

In [32]:
# Make sure dates are of same structure and change data type
df['Date Published'].unique()

array(['20-01-2004', '27-01-2004', '29-01-2004', '27-02-2004',
       '25-02-2004', '25-03-2004', '05-04-2004', '10-04-2004',
       '24-05-2004', '19-09-2004', '28-10-2004', '08-11-2004',
       '28-11-2004', '05-03-2005', '17-03-2005', '15/2/2005',
       '13-05-2005', '27-06-2005', '06-11-2005', '09-11-2005',
       '01-12-2005', '05-12-2005', '13-12-2005', '02-01-2006',
       '12-01-2006', '01-03-2006', '05-03-2006', '12-03-2006',
       '14-03-2006', '28-03-2006', '16-04-2006', '09-05-2006',
       '18-05-2006', '12-06-2006', '22-09-2006', '06-10-2006',
       '09-10-2006', '13-10-2006', '18-10-2006', '02-11-2006',
       '13-11-2006', '16-12-2006', '14-03-2007', '27-06-2007',
       '11-07-2007', '12-07-2007', '20-07-2007', '01-08-2007',
       '26-07-2007', '19-08-2007', '01-09-2007', '19-09-2007',
       '16-10-2007', '23-10-2007', '25-11-2007', '04-01-2008',
       '11-01-2008', '01-05-2008', '09-05-2008', '02-06-2008',
       '13-06-2008', '23-06-2008', '20-06-2008', '22-06-

In [33]:
# Make sure Source are of same structure, and remove redundancy
df['Source'].unique()

array(['Ananoya', 'The Hindu ', 'The Indonesian National News Agency',
       'Wildlife Protection Society of India Press Release', 'The Hindu',
       'Orissa.net', 'NDTV.com', 'Agence France Presse', 'Newkerala.com',
       'Independent Online', 'Jakarta Post', 'Terra Daily',
       'Deepika Global', 'Webindia123', 'The Statesman',
       'Webindia123.com', 'Newindpress.com', 'The Jakarta Post',
       'Reuters', 'Bahrain News Agency', 'Newindpress',
       'New Straits Times', 'Colombo Page', 'Chennai Online', 'Enews.com',
       'Telugu Portal ', 'IANS', 'DNA', 'News.com', 'New Kerala.com',
       'Anandabazar', 'NEWSPost India', 'Fox News', 'Associated Press',
       'Statesman News Service', 'The Himalayan Times',
       'The Associated Press', 'Thaindian News ', 'Times of India',
       'Express News Service', 'The Telegraph', 'Meri News', 'DNA ',
       'The Economic Times', 'Asia-Pacific News', 'Asia News Network',
       'Sri Lanka Daily News', 'VietnamNet', 'Little About',
 

In [34]:
train = df[['Source', 'Title','Date Published', 'Article']]
train['Date Published'] = train['Date Published'].str.replace(',','-')
train['Date Published'] =  pd.to_datetime(train['Date Published'], format="%d-%m-%Y", errors="coerce").fillna(pd.to_datetime(train['Date Published'], format="%Y-%m-%d", errors="coerce"))
train['Date Published'] = train['Date Published'].dt.date
train['Source'] = train['Source'].str.strip()
train['Source'] = train['Source'].str.upper()
train['Article'] = train['Article'].replace('\r','')

/var/folders/g6/8qf633_j27j4pw_gqbrz6dbh0000gn/T/ipykernel_23378/2866659192.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['Date Published'] = train['Date Published'].str.replace(',','-')
/var/folders/g6/8qf633_j27j4pw_gqbrz6dbh0000gn/T/ipykernel_23378/2866659192.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['Date Published'] =  pd.to_datetime(train['Date Published'], format="%d-%m-%Y", errors="coerce").fillna(pd.to_datetime(train['Date Published'], format="%Y-%m-%d", errors="coerce")

In [35]:
train['Source'].unique()

array(['ANANOYA', 'THE HINDU', 'THE INDONESIAN NATIONAL NEWS AGENCY',
       'WILDLIFE PROTECTION SOCIETY OF INDIA PRESS RELEASE', 'ORISSA.NET',
       'NDTV.COM', 'AGENCE FRANCE PRESSE', 'NEWKERALA.COM',
       'INDEPENDENT ONLINE', 'JAKARTA POST', 'TERRA DAILY',
       'DEEPIKA GLOBAL', 'WEBINDIA123', 'THE STATESMAN',
       'WEBINDIA123.COM', 'NEWINDPRESS.COM', 'THE JAKARTA POST',
       'REUTERS', 'BAHRAIN NEWS AGENCY', 'NEWINDPRESS',
       'NEW STRAITS TIMES', 'COLOMBO PAGE', 'CHENNAI ONLINE', 'ENEWS.COM',
       'TELUGU PORTAL', 'IANS', 'DNA', 'NEWS.COM', 'NEW KERALA.COM',
       'ANANDABAZAR', 'NEWSPOST INDIA', 'FOX NEWS', 'ASSOCIATED PRESS',
       'STATESMAN NEWS SERVICE', 'THE HIMALAYAN TIMES',
       'THE ASSOCIATED PRESS', 'THAINDIAN NEWS', 'TIMES OF INDIA',
       'EXPRESS NEWS SERVICE', 'THE TELEGRAPH', 'MERI NEWS',
       'THE ECONOMIC TIMES', 'ASIA-PACIFIC NEWS', 'ASIA NEWS NETWORK',
       'SRI LANKA DAILY NEWS', 'VIETNAMNET', 'LITTLE ABOUT',
       'DAILY INDIA', 'PR

In [36]:
train['Date Published']

0      2004-01-20
1      2004-01-27
2      2004-01-27
3      2004-01-29
4      2004-01-29
          ...    
830    2018-12-28
831    2013-05-21
832    2013-04-09
833    2009-01-28
834    2009-01-28
Name: Date Published, Length: 833, dtype: object

In [37]:
train['Article']

0      Asian elephants electrocuted after drinking session\r\nAnanova\r\nJanuary 20, 2004\r\n\r\nFour wild Asian elephants that got drunk on rice beer have been electrocuted \r\nin the northern Indian state of Meghalaya.\r\n\r\nA forest official said a herd of about 24 elephants went on the rampage in a \r\nremote area in the West Garo Hills district.\r\n\r\n"The elephants, after getting high on rice...
1      2 HELD FOR ELEPHANTS ELECTROCUTION\r\nThe Hindu\r\nJanuary 27, 2004\r\n\r\nTwo farm assistants have been taken into custody by special teams of the \r\nForest department in connection with the electrocution of a tusker and a \r\nfemale elephant at two places in the Coimbatore reserve forest division. One \r\nincident was reported on Saturday night and the other on Sunday. The female \r\nelepha...
2      2 HELD FOR ELEPHANTS ELECTROCUTION\r\nThe Hindu\r\nJanuary 27, 2004\r\n\r\nTwo farm assistants have been taken into custody by special teams of the \r\nForest department in connec

In [38]:
train

,Source,Title,Date Published,Article
0,ANANOYA,Asian elephants electrocuted after drinking session,2004-01-20,"Asian elephants electrocuted after drinking session\r\nAnanova\r\nJanuary 20, 2004\r\n\r\nFour wild Asian elephants that got drunk on rice beer have been electrocuted \r\nin the northern Indian state of Meghalaya.\r\n\r\nA forest official said a herd of about 24 elephants went on the rampage in a \r\nremote area in the West Garo Hills district.\r\n\r\n""The elephants, after getting high on rice..."
1,THE HINDU,2 Held for Elephants electrocution,2004-01-27,"2 HELD FOR ELEPHANTS ELECTROCUTION\r\nThe Hindu\r\nJanuary 27, 2004\r\n\r\nTwo farm assistants have been taken into custody by special teams of the \r\nForest department in connection with the electrocution of a tusker and a \r\nfemale elephant at two places in the Coimbatore reserve forest division. One \r\nincident was reported on Saturday night and the other on Sunday. The female \r\nelepha..."
2,THE HINDU,2 Held for Elephants electrocution,2004-01-27,"2 HELD FOR ELEPHANTS ELECTROCUTION\r\nThe Hindu\r\nJanuary 27, 2004\r\n\r\nTwo farm assistants have been taken into custody by special teams of the \r\nForest department in connection with the electrocution of a tusker and a \r\nfemale elephant at two places in the Coimbatore reserve forest division. One \r\nincident was reported on Saturday night and the other on Sunday. The female \r\nelepha..."
3,THE INDONESIAN NATIONAL NEWS AGENCY,Two Elephants in Riau Poisoned to Death,2004-01-29,"TWO ELEPHANTS IN RIAU POISONED TO DEATH\r\nAntara - The Indonesian National News Agency\r\nJanuary 29, 2004\r\n\r\nPekanbaru, Jan 29 (ANTARA) - Two male elephants were found dead after being \r\npoisoned by unidentified persons at spot near the elephant training center \r\n(PTL) in Duri area, Bengkalis district, Riau, recently.\r\n\r\nChairman of a local non-governmental organization on wildli..."
4,THE INDONESIAN NATIONAL NEWS AGENCY,Experts call for investigation on killing of elephants in Indonesia Province.,2004-01-29,"EXPERTS CALL FOR INVESTIGATION ON KILLING OF ELEPHANTS IN INDON PROVINCE\r\nAntara - The Indonesian National News Agency\r\nJanuary 29, 2004\r\n\r\nBandar Lampung, Lampung, Jan 29 (ANTARA) - A number of environmental \r\nactivists in Indonesia's Lampung province have urged concerned agencies to \r\nlook into the killing of four elephants at the Bukit Barisan National Park, \r\na spokesman has ..."
...,...,...,...,...
830,NEWSMEN,"Spate of Wildlife Deaths Continues in N Bengal, Now Elephant Dies in Siliguri (India)",2018-12-28,"Spate of Wildlife Deaths Continues in N Bengal, Now Elephant Dies in\r\nSiliguri (India)\r\nNewsmen\r\nDecember 28, 2018\r\n\r\nSee link\r\n<https://newsmen.in/news/spate-of-wildlife-deaths-continues-in-n-bengal-now-elephant-dies-in-siliguri/>\r\nfor photo.\r\n\r\nSILIGURI: With the winter cold intensifying, north Bengal is witnessing one\r\nafter another poaching or mystery deaths of wild ani..."
831,DECCAN CHRONICLE,Jumbos electrocuted (India),2013-05-21,"Jumbos electrocuted (India)\r\n\r\nDeccan Chronicle\r\n\r\n21st May 2013\r\n\r\n\r\n\r\nKrishnagiri: Two cow elephants died on Monday after they came into contact with a live Tangedco electric wire that passes through the reserved forest area in Denkannikottai taluk, near Hosur, of Krishnagiri.\r\n\r\n\r\n\r\nOn Monday morning, villagers found the carcass of two female elephants near their vil..."
832,BANGKOK POST,Ivory hunters kill another elephant (Thailand),2013-04-09,"Ivory hunters kill another elephant (Thailand) \r\nBangkok Post \r\n9 Apr 2013 \r\n \r\nThe dessicated carcass of a dead elephant has been found in Kaeng Krachan National Park, in Phetchaburi province, about three kilometres from where another elephant was found dead on March 8. \r\n \r\nChaiwat Limlikhit-aksorn, the park chief, said he was informed of the find by villagers on Tuesday morning ..."
833,TIMES OF INDIA,Two elephants die in Mysore district,2009-01-28,"Two ele

# Construct data to be predicted

Going to start off by seperating each objective into its own set, one dataframe for Country, one for Total, and etc.

In [39]:
test_country = df['Country']
test_state = df['State']
test_district = df['District']
test_village = df['Village/Place']
test_total = df['Total']
test_date = df['Date (Event)']

In [40]:
test_country = test_country.str.strip()
test_country.unique()

array(['India', 'Indonesia', 'Bangladesh', 'Malaysia', 'Sri Lanka',
       'Nepal', 'Thailand', 'Vietnam', 'Laos', 'China', 'Cambodia',
       'Burma'], dtype=object)

In [41]:
test_total.unique()

array(['24', '1', '2', '4', '10', '6', 'NG', '13', '3', '25', '9', '17',
       '40', '80', '5', '7', '50', '60', '21', '8', '16', '15', '35',
       '22', '30', '12', '19', '55'], dtype=object)

In [42]:
test_village.nunique()

676

In [43]:
test_village = test_village.str.strip()
test_village = test_village.str.upper()
#test_village.unique()

In [44]:
test_village.nunique()

668

In [45]:
test_district = test_district.str.strip()
test_district = test_district.str.upper()
test_district.unique()

array(['WEST GARO HILLS', 'COIMBATORE', 'NG', 'HARIDWAR',
       'PATHANAMTHITTA', 'MYSORE', 'KEONJHAR', 'COX’S BAZAAR', 'KHUNTI',
       'DUMKA', 'NAINITAL', 'JALPAIGURI', 'PAURI GHARWAL', 'UDALGURI',
       'SERAIKELA KHARSAWAN', 'THENI', 'MAYURBHANJ', 'KHAJARA',
       'DINDIGUL', 'SIVAGIRI FOREST RANGE IN TIRUNELVELI', 'DEHRADUN',
       'JAJAPUR', 'NILIGIRIS', 'RANCHI', 'SAMBALPUR', 'KRISHNAGIRI',
       'SONITPUR', 'BANGALORE', 'DOOARS', 'DARJEELING', 'VIZIANAGARAM',
       'COORG', 'JAMSHEDPUR', 'PALAKKAD', 'NAGAON', 'ERODE', 'SUNDERGARH',
       'KHUTI', 'CHANDGAD', 'SIVASAGAR', 'SINGHBHUM', 'KHAO CHAMAO',
       'KODAGU', 'CHAMARAJNAGAR', 'KARBI ANGLONG', 'KENDUJHAR', 'ANGUL',
       'KHURDA KHORDA', 'SAHARANPUR', 'BALASORE', 'MIDNAPORE',
       'KAMRUP METROPOLITAN', 'GOLAGHAT', 'IDUKKI', 'MORIGAON',
       'NAYAGRAM', 'DHENKANAL', 'GUMLA', 'HASSAN', 'WEST MIDNAPORE',
       'BALANGIR', 'LAKHIMPUR', 'GANJAM', 'PALAMU', 'JORHAT',
       'CHAMARAJANAGAR', 'BARGARH', 'SOUTH 24 P

In [46]:
test_state = test_state.str.strip()
test_state = test_state.str.upper()
test_state.unique()

array(['MEGHALAYA', 'TAMIL NADU', 'RIAU', 'WEST LAMPUNG', 'UTTARAKHAND',
       'KERALA', 'KARNATAKA', 'ODISHA', 'NG', 'JHARKHAND', 'JAKARTA',
       'SABAH', 'WEST BENGAL', 'ASSAM', 'CHHATTISGARH', 'MADHYA PRADESH',
       "COX'S BAZAR", 'HULU PERAK', 'POLONNARUWA', 'JHAPA',
       'ANDHRA PRADESH', 'LALMONIRHAT', 'MAHARASHTRA', 'RAYONG',
       'SUMATRA', 'ANURADHAPURA', 'VINH CUU', 'LAMPUNG', 'UTTAR PRADESH',
       'NAXAITHONG', 'EA SUP', 'TAMIANG', 'TRINCOMALEE', 'DELHI',
       'KANCHANADIT', 'BUON DON', 'YUNNAN', 'THANAMALVILA', 'BENGKALIS',
       'DAK LAK', 'BAKAMUNA', 'ACEH', 'ACEH JAYA', 'EAST ACEH',
       'BO PHLOI', 'MYMENSINGH', 'SI SAWAT', 'BANJARNEGARA',
       'PHETCHABURI', "PHETCHABURI'S", 'HAMBANTOTA', 'KAENG KRACHAN',
       'RIAUA', 'MANNAR', 'WEST ACEH', 'PRACHUAP KHIRI KHAN', 'PELALAWAN',
       'TAWAU', 'MONDULKIRI', 'PARSA', 'KURUNEGALA', 'ARUNACHAL PRADESH',
       'MURSHIDABAD', 'DAL LAK', 'MUANG', 'PUTTALAM', 'JAYA',
       'KANCHANABURI', 'NORTH CENTRAL P

In [47]:
test_date.unique()

array(['20-01-2004', '24-01-2004', '25-01-2004', '29-01-2004',
       '20-02-2004', '23-02-2004', '24-03-2004', '29-03-2004',
       '02-04-2004', '08-04-2004', '21-05-2004', '19-09-2004',
       '28-10-2004', '08-11-2004', '28-11-2004', '05-03-2005',
       '17-03-2005', '02-04-2005', '13-05-2005', '27-06-2005',
       '30-09-2005', '06-11-2005', '01-12-2005', '02-12-2005',
       '13-12-2005', '02-01-2006', '12-01-2006', '01-03-2006',
       '12-03-2006', '14-03-2006', '28-03-2006', '16-04-2006',
       '09-05-2006', '18-05-2006', '12-06-2006', '22-09-2006',
       '06-10-2006', '09-10-2006', '13-10-2006', '18-10-2006',
       '02-11-2006', '13-11-2006', '16-12-2006', '14-03-2007',
       '27-06-2007', '11-07-2007', '12-07-2007', '20-07-2007',
       '24-07-2007', '26-07-2007', '30-07-2007', '10-08-2007',
       '01-09-2007', '19-09-2007', '16-10-2007', '23-10-2007',
       '25-11-2007', '04-01-2008', '11-01-2008', '01-05-2008',
       '08-05-2008', '02-06-2008', '13-06-2008', '18-06

In [48]:
test_date = test_date.str.replace('/','-')
test_date.unique()

array(['20-01-2004', '24-01-2004', '25-01-2004', '29-01-2004',
       '20-02-2004', '23-02-2004', '24-03-2004', '29-03-2004',
       '02-04-2004', '08-04-2004', '21-05-2004', '19-09-2004',
       '28-10-2004', '08-11-2004', '28-11-2004', '05-03-2005',
       '17-03-2005', '02-04-2005', '13-05-2005', '27-06-2005',
       '30-09-2005', '06-11-2005', '01-12-2005', '02-12-2005',
       '13-12-2005', '02-01-2006', '12-01-2006', '01-03-2006',
       '12-03-2006', '14-03-2006', '28-03-2006', '16-04-2006',
       '09-05-2006', '18-05-2006', '12-06-2006', '22-09-2006',
       '06-10-2006', '09-10-2006', '13-10-2006', '18-10-2006',
       '02-11-2006', '13-11-2006', '16-12-2006', '14-03-2007',
       '27-06-2007', '11-07-2007', '12-07-2007', '20-07-2007',
       '24-07-2007', '26-07-2007', '30-07-2007', '10-08-2007',
       '01-09-2007', '19-09-2007', '16-10-2007', '23-10-2007',
       '25-11-2007', '04-01-2008', '11-01-2008', '01-05-2008',
       '08-05-2008', '02-06-2008', '13-06-2008', '18-06

In [49]:
test_date =  pd.to_datetime(test_date, format="%d-%m-%Y", errors="coerce").fillna(pd.to_datetime(test_date, format="%Y-%m-%d", errors="coerce"))
test_date = test_date.dt.date
test_date

0      2004-01-20
1      2004-01-24
2      2004-01-25
3      2004-01-29
4      2004-01-29
          ...    
830    2018-12-28
831    2013-05-20
832    2013-03-23
833    2009-01-26
834    2009-01-26
Name: Date (Event), Length: 833, dtype: object

# Common Words for search
https://stackoverflow.com/questions/2452982/how-to-extract-common-significant-phrases-from-a-series-of-text-entries

In [50]:
common_words = ['it', 'is', 'not', 'a', 'but', 'on', 'the', 'in', 'all', 'ways','of', 'to', 'and', 'was', 'by', 'that', 'for', 'at', 'were', 'an', 'have', 'said','are', 'been', 'had', 'with', 'from', 'as', 'he', 'has', 'its', 'be', 'they', 'after','one','two','three','four','1','2','3','4','male','female']
common_set = set(common_words)

In [51]:
title_eda = train['Title'].replace(to_replace='\(', value="", regex=True)
title_eda = title_eda.replace(to_replace='\)', value="", regex=True)
title_eda = title_eda.replace(to_replace='\,', value="", regex=True)

In [52]:
frequent_title = pd.Series(' '.join(title_eda).lower().split()).value_counts()[:70]
frequent_title[[s for s in frequent_title.index if not s in common_set]]

elephant         446
india            399
found            183
dead             164
electrocuted     152
elephants        147
tusker           102
wild              93
state             87
dies              86
jumbo             84
district          71
forest            64
killed            58
calf              57
death             54
sri               53
lanka             51
carcass           40
odisha            39
poachers          39
indonesia         36
jumbos            35
assam             35
shot              34
tusks             34
another           32
near              29
thailand          25
injured           25
tamil             24
nadu              24
electrocution     23
sumatran          22
suspected         21
officials         21
deaths            20
kill              20
electric          19
kills             19
wildlife          17
poisoned          17
villagers         16
into              16
malaysia          15
reserve           15
ivory             15
jharkhand    

# Saving as .txt for google cloud auto ml

In [55]:
i = 0
for row in train['Article']:
    if i > len(train['Article']):
       break
    else:
       f = open(str(i)+'.txt', 'w')
       f.write(row)
       f.close()
       i+=1

# Import various news data to add to training data

In [ ]:
!unzip /content/drive/MyDrive/all-the-news-2-1.zip -d /content/
#all_news = pd.read_csv("/content/all-the-news-2-1.csv", sep=',', usecols=['publication','article','date','title'])
all_news = pd.read_csv("/content/all-the-news-2-1.csv", sep=',', usecols=['article'])

Archive:  /content/drive/MyDrive/all-the-news-2-1.zip
  inflating: /content/all-the-news-2-1.csv  


In [ ]:
#all_news['date'] =  pd.to_datetime(all_news['date'], format="%Y-%m-%d %H:%M:%S", errors="coerce")
#all_news['date'] = all_news['date'].dt.date
#all_news.rename(columns={'date': 'Date Published', 'title': 'Title', 'article': 'Article', 'publication': 'Source', }, inplace=True)
#all_news.rename(columns={'title': 'Title', 'article': 'Article'}, inplace=True)
all_news.rename(columns={'article': 'Article'}, inplace=True)
all_news.head()

,Article
0,"This post is part of Polyarchy, an independent blog produced by the political reform program at New America, a Washington think tank devoted to developing new ideas and new voices. Imagine you are an otherwise healthy 30-something who starts feeling weird. You are sometimes short of breath. You get migraines. Your feet start to swell a little. But otherwise, everything seems fine. You go to th..."
1,"The Indianapolis Colts made Andrew Luck the highest-paid player in NFL history this offseason with a five-year, $122-million contract with $89 million guaranteed. However, they're already finding that Luck's contract is inhibiting their ability to address weaknesses on other parts of the roster, particularly on defense. On Friday, Colts GM Ryan Grigson, who is under fire for the Colts 1-3 sta..."
2,"DAVOS, Switzerland (Reuters) - U.S. President Donald Trump denied a report on Friday that he had ordered Special Counsel Robert Mueller fired last June, calling it “fake news”. The New York Times reported on Thursday that Trump backed down from his order after the White House counsel threatened to resign rather than follow his directive, citing four people told of the matter. “Fake news, fol..."
3,"PARIS (Reuters) - Former French president Nicolas Sarkozy published a new memoir on Thursday but was quick to dismiss speculation he might return to politics to rescue his old center-right party, which has shed support since President Emmanuel Macron stormed to power. Entitled “Passions”, the autobiography documents Sarkozy’s rise to power, while hurling barbs at some of his former allies, in..."
4,"Paris Hilton arrived at LAX Wednesday dressed to pay her last respects to her uncle Monty Brinson. Paris flew in from Switzerland especially for the funeral of Brinson, who used to be married to her aunt Kim Richards. Monty died Sunday after a long battle with cancer ... and the loss is obviously hitting Paris hard. She posted a picture collage with Monty calling him an ""incredible man with su..."


In [ ]:
all_news['Y'] = 0
all_news.head()

,Article,Y
0,"This post is part of Polyarchy, an independent blog produced by the political reform program at New America, a Washington think tank devoted to developing new ideas and new voices. Imagine you are an otherwise healthy 30-something who starts feeling weird. You are sometimes short of breath. You get migraines. Your feet start to swell a little. But otherwise, everything seems fine. You go to th...",0
1,"The Indianapolis Colts made Andrew Luck the highest-paid player in NFL history this offseason with a five-year, $122-million contract with $89 million guaranteed. However, they're already finding that Luck's contract is inhibiting their ability to address weaknesses on other parts of the roster, particularly on defense. On Friday, Colts GM Ryan Grigson, who is under fire for the Colts 1-3 sta...",0
2,"DAVOS, Switzerland (Reuters) - U.S. President Donald Trump denied a report on Friday that he had ordered Special Counsel Robert Mueller fired last June, calling it “fake news”. The New York Times reported on Thursday that Trump backed down from his order after the White House counsel threatened to resign rather than follow his directive, citing four people told of the matter. “Fake news, fol...",0
3,"PARIS (Reuters) - Former French president Nicolas Sarkozy published a new memoir on Thursday but was quick to dismiss speculation he might return to politics to rescue his old center-right party, which has shed support since President Emmanuel Macron stormed to power. Entitled “Passions”, the autobiography documents Sarkozy’s rise to power, while hurling barbs at some of his former allies, in...",0
4,"Paris Hilton arrived at LAX Wednesday dressed to pay her last respects to her uncle Monty Brinson. Paris flew in from Switzerland especially for the funeral of Brinson, who used to be married to her aunt Kim Richards. Monty died Sunday after a long battle with cancer ... and the loss is obviously hitting Paris hard. She posted a picture collage with Monty calling him an ""incredible man with su...",0


In [ ]:
train['Y'] = 1
train.drop(['Source', 'Date Published', 'Title'], axis=1, inplace=True)
train.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Article,Y
0,"Asian elephants electrocuted after drinking session\r\nAnanova\r\nJanuary 20, 2004\r\n\r\nFour wild Asian elephants that got drunk on rice beer have been electrocuted \r\nin the northern Indian state of Meghalaya.\r\n\r\nA forest official said a herd of about 24 elephants went on the rampage in a \r\nremote area in the West Garo Hills district.\r\n\r\n""The elephants, after getting high on rice...",1
1,"2 HELD FOR ELEPHANTS ELECTROCUTION\r\nThe Hindu\r\nJanuary 27, 2004\r\n\r\nTwo farm assistants have been taken into custody by special teams of the \r\nForest department in connection with the electrocution of a tusker and a \r\nfemale elephant at two places in the Coimbatore reserve forest division. One \r\nincident was reported on Saturday night and the other on Sunday. The female \r\nelepha...",1
2,"2 HELD FOR ELEPHANTS ELECTROCUTION\r\nThe Hindu\r\nJanuary 27, 2004\r\n\r\nTwo farm assistants have been taken into custody by special teams of the \r\nForest department in connection with the electrocution of a tusker and a \r\nfemale elephant at two places in the Coimbatore reserve forest division. One \r\nincident was reported on Saturday night and the other on Sunday. The female \r\nelepha...",1
3,"TWO ELEPHANTS IN RIAU POISONED TO DEATH\r\nAntara - The Indonesian National News Agency\r\nJanuary 29, 2004\r\n\r\nPekanbaru, Jan 29 (ANTARA) - Two male elephants were found dead after being \r\npoisoned by unidentified persons at spot near the elephant training center \r\n(PTL) in Duri area, Bengkalis district, Riau, recently.\r\n\r\nChairman of a local non-governmental organization on wildli...",1
4,"EXPERTS CALL FOR INVESTIGATION ON KILLING OF ELEPHANTS IN INDON PROVINCE\r\nAntara - The Indonesian National News Agency\r\nJanuary 29, 2004\r\n\r\nBandar Lampung, Lampung, Jan 29 (ANTARA) - A number of environmental \r\nactivists in Indonesia's Lampung province have urged concerned agencies to \r\nlook into the killing of four elephants at the Bukit Barisan National Park, \r\na spokesman has ...",1


In [ ]:
train = pd.concat([train, all_news])
del all_news
gc.collect()
train.head()

,Article,Y
0,"Asian elephants electrocuted after drinking session\r\nAnanova\r\nJanuary 20, 2004\r\n\r\nFour wild Asian elephants that got drunk on rice beer have been electrocuted \r\nin the northern Indian state of Meghalaya.\r\n\r\nA forest official said a herd of about 24 elephants went on the rampage in a \r\nremote area in the West Garo Hills district.\r\n\r\n""The elephants, after getting high on rice...",1
1,"2 HELD FOR ELEPHANTS ELECTROCUTION\r\nThe Hindu\r\nJanuary 27, 2004\r\n\r\nTwo farm assistants have been taken into custody by special teams of the \r\nForest department in connection with the electrocution of a tusker and a \r\nfemale elephant at two places in the Coimbatore reserve forest division. One \r\nincident was reported on Saturday night and the other on Sunday. The female \r\nelepha...",1
2,"2 HELD FOR ELEPHANTS ELECTROCUTION\r\nThe Hindu\r\nJanuary 27, 2004\r\n\r\nTwo farm assistants have been taken into custody by special teams of the \r\nForest department in connection with the electrocution of a tusker and a \r\nfemale elephant at two places in the Coimbatore reserve forest division. One \r\nincident was reported on Saturday night and the other on Sunday. The female \r\nelepha...",1
3,"TWO ELEPHANTS IN RIAU POISONED TO DEATH\r\nAntara - The Indonesian National News Agency\r\nJanuary 29, 2004\r\n\r\nPekanbaru, Jan 29 (ANTARA) - Two male elephants were found dead after being \r\npoisoned by unidentified persons at spot near the elephant training center \r\n(PTL) in Duri area, Bengkalis district, Riau, recently.\r\n\r\nChairman of a local non-governmental organization on wildli...",1
4,"EXPERTS CALL FOR INVESTIGATION ON KILLING OF ELEPHANTS IN INDON PROVINCE\r\nAntara - The Indonesian National News Agency\r\nJanuary 29, 2004\r\n\r\nBandar Lampung, Lampung, Jan 29 (ANTARA) - A number of environmental \r\nactivists in Indonesia's Lampung province have urged concerned agencies to \r\nlook into the killing of four elephants at the Bukit Barisan National Park, \r\na spokesman has ...",1


In [ ]:
train = train[train.Article.notna()]
train = train[train.Y.notna()]

In [ ]:
train.isnull().values.any()

False

In [ ]:
train[train.Article.isnull()]

,Article,Y


# Validation Set

In [ ]:
def split_data(df, test_percentage,cv):
  train, test = train_test_split(df, test_size=int(test_percentage*len(df)), stratify = df["Y"])
  val = None
  if cv == True:
    train, val = train_test_split(df, test_size=int(test_percentage*len(df)), stratify = df["Y"])
  return train, test, val

In [ ]:
train, test, val = split_data(train, 0.2, True)
train_y = train['Y'].to_numpy()
test_y = test['Y'].to_numpy()
val_y = val['Y'].to_numpy()
train.drop(columns='Y', inplace=True)
test.drop(columns='Y', inplace=True)
val.drop(columns='Y', inplace=True)
train =  train.to_numpy()
test =  test.to_numpy()
val =  val.to_numpy()

# Trying out NLP
*   https://www.tensorflow.org/hub/tutorials/tf2_text_classification#create_a_validation_set
*   https://www.tensorflow.org/hub/tutorials/bert_experts
*  https://www.tensorflow.org/hub/tutorials/semantic_similarity_with_tf_hub_universal_encoder
* https://www.tensorflow.org/hub/tutorials/semantic_similarity_with_tf_hub_universal_encoder

## Encoding to help represent text as vectors for input to a model
Tried out spacy and tensorflow

In [ ]:
#spacy.cli.download("en_core_web_trf")
#nlp = spacy.load("en_core_web_trf")

In [ ]:
#doc = nlp(train['Article'][1])
#displacy.render(doc, style='ent', jupyter=True)

In [ ]:
train = train.squeeze()
train

array(['WASHINGTON, May 17 (Reuters) - U.S. government officials on Thursday are set to begin investigating Boeing Co’s unfair trade claims against Canadian rival Bombardier, a two-track action that could lead to U.S. duties on Bombardier’s new jetliner and also pits Boeing against Delta Air Lines Inc. The U.S. Commerce Department is poised to announce the launch of its investigation. U.S. International Trade Commission (USITC) staff will hear testimony from both companies and from Bombardier customer Delta in the case. Boeing alleges that Bombardier’s new 100-150 seat CSeries jetliners are being dumped below cost in the U.S. market, and are unfairly subsidized by Canadian taxpayers. The new Canadian aircraft competes with Boeing’s 737-700 and 737 MAX 7 jets. Delta agreed last year to buy dozens of CSeries planes at a price that Boeing says was well below Bombardier’s cost. The case has increased trade tensions between the United States and Canada - along with disputes over Canadian so

In [ ]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")
embed_layer = hub.KerasLayer(embed, input_shape=[], dtype=tf.string, trainable=True)
embed_layer(train)

In [ ]:
model = tf.keras.Sequential()
model.add(embed_layer)
#model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))
model.summary()
model.compile(optimizer='adam',
              loss=tf.losses.BinaryCrossentropy(from_logits=True),
              metrics=[tf.metrics.BinaryAccuracy(threshold=0.0, name='accuracy')])
history = model.fit(train,
                    train_y,
                    epochs=1,
                    batch_size=10,
                    validation_data=(val,val_y),
                    verbose=1)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 512)               147354880 
                                                                 
 dense (Dense)               (None, 1)                 513       
                                                                 
Total params: 147,355,393
Trainable params: 147,355,393
Non-trainable params: 0
_________________________________________________________________


In [ ]:
print("Input")
[print(i.shape, i.dtype) for i in model.inputs]
print("Out")
[print(o.shape, o.dtype) for o in model.outputs]
print("---")
[print(l.name, l.input_shape, l.dtype) for l in model.layers]